In [1]:
1#dataset.py (adjust for R(2+1)D)
import os
import torch
from torch.utils.data import Dataset
from load_video_test import load_video

class CricketVideoDataset(Dataset):
    def _init_(self, root_dir, max_frames=64):
        self.root_dir = root_dir
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.video_paths = []
        self.labels = []

        for cls in self.classes:
            cls_path = os.path.join(root_dir, cls)
            for file in os.listdir(cls_path):
                if file.endswith(".mp4"):
                    self.video_paths.append(os.path.join(cls_path, file))
                    self.labels.append(self.class_to_idx[cls])

        self.max_frames = max_frames

    def _len_(self):
        return len(self.video_paths)

    def _getitem_(self, idx):
        video_path = self.video_paths[idx]
        label = self.labels[idx]
        video_tensor = load_video(video_path, self.max_frames)
        return video_tensor, torch.tensor(label)

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class R2Plus1D(nn.Module):
    def __init__(self, num_classes=10):
        super(R2Plus1D, self).__init__()

        # Example: first conv block (2D + 1D decomposition)
        self.conv1_spatial = nn.Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3))
        self.conv1_temporal = nn.Conv3d(64, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
        self.bn1 = nn.BatchNorm3d(64)

        # More layers can be added (ResNet-style)
        self.pool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        # Input shape: (B, 3, T, H, W)
        x = F.relu(self.bn1(self.conv1_temporal(self.conv1_spatial(x))))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


In [6]:
import torch
from torch.utils.data import DataLoader
from dataset import CricketVideoDataset
from model import R2Plus1D


# Dataset path
train_dir = r"C:\Users\singh\pratap\cric proj\cricket_dataset\train"
val_dir = r"C:\Users\singh\pratap\cric proj\cricket_dataset\val"

# Hyperparameters
num_classes = 3
batch_size = 2
num_epochs = 10
learning_rate = 1e-4
max_frames = 64

# Dataset & DataLoader
train_dataset = CricketVideoDataset(train_dir, max_frames=max_frames)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Model, Loss, Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = R2Plus1D_Cricket(num_classes=num_classes).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for videos, labels in train_loader:
        videos = videos.permute(0, 2, 1, 3, 4).to(device)  # (B, T, C, H, W) -> (B, C, T, H, W)
        labels = labels.to(device)

        outputs = model(videos)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {100*correct/total:.2f}%")

# Save model
torch.save(model.state_dict(), "weights/r2plus1d_cricket.pth")
print("✅ Model saved successfully.")

ImportError: cannot import name 'R2Plus1D' from 'model' (c:\Users\sanvi\OneDrive\Desktop\main_project\Action_Recognition_for_cricket_commentary\cric proj\model.py)